# RDP access functions

In [1]:
import pandas as pd
import requests, json, math, time, gzip, datetime

In [8]:
APP_KEY                     = 'YOUR_APP_KEY'
RDP_LOGIN                   = 'YOUR_PLATFORM_LOGIN'
RDP_PASSWORD                = 'YOUR_PLATFORM_PASSWORD'

### Get Token (Internal)

In [3]:
tknObject = {}

def getToken():
    global tknObject
    if 'expiry_tm' in tknObject and tknObject['expiry_tm'] > time.time():
        return tknObject["access_token"]

    print('Getting access token...')
    if 'refresh_token' in tknObject:
        tData = {
            'refresh_token': tknObject['refresh_token'],
            'grant_type': "refresh_token",
        }
    else:
        tData = {
            'username': RDP_LOGIN,
            'password': RDP_PASSWORD,
            'grant_type': 'password',
            'scope': 'trapi',
            'takeExclusiveSignOnControl': 'true'
        }
    
    response = requests.post('https://api.refinitiv.com/auth/oauth2/v1/token',
        headers = {'Accept': 'application/json'},
        data = tData,
        auth = (APP_KEY, ''))

    if response.status_code != 200:
        raise Exception('Failed to get access token {0} - {1}'.format(response.status_code, response.text))

    print('...token received')
    tknObject = json.loads(response.text)
    tknObject["expiry_tm"] = time.time() + int(tknObject["expires_in"]) - 10
    return tknObject['access_token']


### Get Request

In [4]:
def getRequest(URI, requestData = {}):
    dResp = requests.get('https://api.refinitiv.com' + URI, headers = {'Authorization': 'Bearer ' + getToken()}, params = requestData)

    if dResp.status_code != 200:
        raise Exception('Request failed {0} - {1}'.format(dResp.status_code, dResp.text))
    else:
        jResp = json.loads(dResp.text)
        return jResp 

In [ ]:
def downloadUncompressFile(URI, requestData = {}):
    dResp = requests.get('https://api.refinitiv.com' + URI, headers = {'Authorization': 'Bearer ' + getToken()}, params = requestData, stream = True)

    if dResp.status_code != 200:
        raise Exception('Request failed {0} - {1}'.format(dResp.status_code, dResp.text))
    
    dResp.raw.decode_content = True

    gzip_file = gzip.GzipFile(fileobj=dResp.raw)
    fBytes = gzip_file.read()
    return fBytes.decode('utf8')

### Post Request

In [5]:
def postRequest(URI, requestData):
    dResp = requests.post('https://api.refinitiv.com' + URI, headers = {'Authorization': 'Bearer ' + getToken(),  'Content-Type': 'application/json'}, data = json.dumps(requestData))
    if dResp.status_code != 200:
        raise Exception('Request failed {0} - {1}'.format(dResp.status_code, dResp.text))
    else:
        jResp = json.loads(dResp.text)
        return jResp

### Define the file download function

In [ ]:
buckets = {}
# define the download function
def downloadJSONBulkFile(bucketName, fileAttributes, fileNameKeywords):
    link = f'/file-store/v1/file-sets?bucket={bucketName}&pageSize=100&attributes={fileAttributes}'

    while True:
        # get a list of all the buckets
        hResp = getRequest(link)
        print(hResp)

        # loop through all the buckets
        for bucket in hResp['value']:
            bName = bucket['name']
            # does bucket contains all the matching keywords
            if all([x in bName for x in fileNameKeywords]):
                fileName = bucket['files']
                buckets[bName] = fileName
                print('Found bucket: ', bName, ', Files: ', fileName)

        # is there pagination of list
        if '@nextLink' in hResp:
            link = hResp['@nextLink']
        else:
            break

    if not buckets:
        raise Exception(f'No matching bulk file found in bucket: {bucketName}')

    # select the bucket with most recent datetime
    selBucket = max(buckets)
    print(f'Downloading - {selBucket}')
    selFiles = buckets[selBucket]
    # download and uncompress the files in this bucket
    fileStrs = []
    for bFile in selFiles:
        fileStr = downloadUncompressFile('/file-store/v1/files/' + bFile + '/stream')
        print(f'File {bFile} downloaded and uncompressed, size: {len(fileStr)}')
        fileStrs.append(fileStr)
    return fileStrs